In [6]:
import requests
from bs4 import BeautifulSoup
import time
import re




access_token = open("/Users/daniellancet/Desktop/Projects/Markov_Chain_Beginner_Project/API_Key.txt").readline() 

# Base URL and headers for Genius API
base_url = 'https://api.genius.com'
headers = {'Authorization': f'Bearer {access_token}'}


In [7]:

def get_song_ids(artist_name, max_results=10):
    search_url = f'{base_url}/search'
    params = {'q': artist_name, 'page': 1, 'per_page': max_results}
    
    song_ids = []
    page = 1
    
    while True:
        params['page'] = page
        response = requests.get(search_url, headers=headers, params=params).json()
        hits = response['response']['hits']
        
        if not hits:
            break
        
        for hit in hits:
            song_id = hit['result']['id']
            song_ids.append(song_id)
        
        if len(hits) < max_results:
            break
        
        page += 1
    
    return song_ids

def get_song_urls(song_ids):
    song_urls = []
    for song_id in song_ids:
        song_url = f'{base_url}/songs/{song_id}'
        response = requests.get(song_url, headers=headers).json()
        lyrics_path = response['response']['song']['path']
        lyrics_url = f'https://genius.com{lyrics_path}'
        song_urls.append(lyrics_url)
    return song_urls

    
def get_song_lyrics(song_url):
    response = requests.get(song_url)
    
    if response.status_code != 200:
        return f"Failed to fetch lyrics page: {response.status_code}"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find the div with data-lyrics-container attribute
    lyrics_divs = soup.find_all('div', attrs={'data-lyrics-container': 'true'})
    
    if not lyrics_divs:
        return "Lyrics not found on the page."
    
    # Combine the text from all such divs (there might be multiple)
    lyrics = "\n".join([div.get_text(separator="\n", strip=True) for div in lyrics_divs])
    lyrics  = lyrics.replace('\n', ' ')
    lyrics = re.sub(r'\[.*?\]', '  ', lyrics)
    
    return lyrics


def get_song_lyrics_txt(song_url, output_file):
    response = requests.get(song_url)
    
    if response.status_code != 200:
        return f"Failed to fetch lyrics page: {response.status_code}"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    

    lyrics_divs = soup.find_all('div', attrs={'data-lyrics-container': 'true'})
    
    if not lyrics_divs:
        return "Lyrics not found on the page."
    
    
    lyrics = "\n".join([div.get_text(separator="\n", strip=True) for div in lyrics_divs])
    lyrics = re.sub(r'\[.*?\]', '  ', lyrics)
    
    
    
    with open(output_file, 'w', encoding='utf-8') as file:
        file.write(lyrics)
    
    return f"Lyrics successfully written to {output_file}"

In [8]:
sample_url = "https://genius.com/Ed-sheeran-thinking-out-loud-lyrics"
sample_lyrics = get_song_lyrics_txt(sample_url, "Song-Lyrics/thinking_out_loud.txt")

In [9]:
def get_artist_songs(artist_name, max_res=10):
    song_ids = get_song_ids(artist_name, max_res)
    song_urls = get_song_urls(song_ids)
    artist_name = artist_name.replace(" ", "-")
    for id, url in zip(song_ids, song_urls):
        
        path = f"Song-Lyrics/{artist_name}/{str(id)}.txt"
        get_song_lyrics_txt(url, path)


#get_artist_songs("Kendrick Lamar", 5)